### Evaluation of Tool Patch Generation  for SmartBugs curated
Notebook to study the patches generated by APR tools on smart contracts

The dataframes generated only calculated patches generated, this does not mean the patch is different from the original contract, neither if they compile nor resolve the vulnerability.

By convention, the result of experiments are allocated on:
'../results/smartbugs/TOOL/CATEGORY/SOL_FILE/Patch.sol'

And the dataset info is declared at 
'../results/smartbugs/contracts.csv'

The results of this notebook will be allocated on:
'../results/smartbugs/'


In [280]:
import os

#collecting contracts name
dataset='smartbugs'
results_path='../results'
contracts_path=os.path.join(results_path,dataset,'contracts.csv')
storage_path=os.path.join(results_path,dataset)

import pandas as pd
contracts_df = pd.read_csv(contracts_path)


In [281]:
def search_contract_on_tool(folder_path, tool_name, category, file_name):
    contract_folder=file_name[:-4]
    tool_folder_path = os.path.join(folder_path, tool_name,category)
    patch_folder = os.path.join(tool_folder_path, contract_folder)
    if not os.path.exists(patch_folder):
        return False  #Means that the tool did not handle the contract
    if tool_name in ["Aroc", "TIPS"]:
        # We asume there is a patch if the folder contains a .sol file
        for file in os.listdir(patch_folder):
            if ('.sol' in file[-4:]):
                return True
        return False #We have not set how to read the results
    #if tool_name in ["TIPS"]:
        ## For patch generation, this tool can follow at least 3 different name conventions UX, R or .sol
        ##For effective counting consider searching for name compatibility and [:-4]== .sol
        ## For future analysis Ux means candidate, R is for reentrancy, and contract.sol means no bug, yet the tool apply some "magic"
        
     #   return "WIP"
    if tool_name in ["Elysium","SmartShield"]:
        #patch is binary
        file_name=contract_folder+'.bin'
        file_path = os.path.join(tool_folder_path, contract_folder,file_name)
        return os.path.exists(file_path)
    file_path = os.path.join(tool_folder_path, contract_folder,file_name)
    return os.path.exists(file_path)

In [282]:
#Test the function to search for contracts
assert search_contract_on_tool(storage_path,"Elysium","access_control",'FibonacciBalance.sol') == True
assert search_contract_on_tool(storage_path,"Aroc","time_manipulation",'lottopollo.sol') == False
assert search_contract_on_tool(storage_path,"Aroc","arithmetic",'integer_overflow_1.sol') == True
assert search_contract_on_tool(storage_path,"sGuardPlus","denial_of_service",'auction.sol') == False
assert search_contract_on_tool(storage_path,"SmartFix","access_control",'FibonacciBalance.sol') == True


In [283]:
folder_path= os.path.join(results_path,dataset)
print(folder_path)

../results/smartbugs


In [284]:
tools = [
    "SmartShield",
    "sGuard",
    "Aroc",
    "Elysium",
    "TIPS",
    "SmartFix",
    "sGuardPlus"
]

In [285]:
['Contract', 'Category', 'SmartShield', 'sGuard', 'Aroc', 'Elysium', 'TIPS', 'SmartFix', 'sGuardPlus']
data_title= ["Contract", "Category"] + tools

In [286]:
def generate_df():
    tool_per_contract=[]
    for contract , category in contracts_df[["Name","Category"]].values.tolist():
        #print(contract)
        #print(category)
        new_line=[contract,category]
        for tool in tools:
            patch_exist=search_contract_on_tool(folder_path, tool, category, contract)
            new_line.append(patch_exist)
        tool_per_contract.append(new_line)
    df= pd.DataFrame(tool_per_contract)
    df.columns=data_title
    return df
        
            
            

In [287]:
df=generate_df()

In [288]:
assert len(df[df['SmartShield']==True].values.tolist()) == 134
assert len(df[df['sGuard']==True].values.tolist()) == 109
assert len(df[df['Aroc']==True].values.tolist()) == 90
assert len(df[df['Elysium']==True].values.tolist()) == 126
assert len(df[df['TIPS']==True].values.tolist()) == 141
assert len(df[df['SmartFix']==True].values.tolist()) == 86
assert len(df[df['sGuardPlus']==True].values.tolist()) == 81


In [290]:
df=generate_df()
#file_name='tool_patch_ability_GEN_per_contract.csv'
#df.to_csv(os.path.join(storage_path,file_name), index=False)  

In [291]:
df=generate_df()
df= df.sort_values(by='Contract', ascending=True)
# Columns to consider
columns_to_consider = ['SmartShield', 'sGuard', "Aroc", 'Elysium', 'TIPS', "SmartFix",'sGuardPlus']

# Summing up the number of True values in each row
df['PatchCount'] = df[columns_to_consider].sum(axis=1)

# Sorting the DataFrame based on the PatchCount column in ascending order
df_sorted = df.sort_values(by='PatchCount', ascending=True)



In [294]:
df_sorted
file_name='tool_patch_ability_GEN_per_contract.csv'
df_sorted.to_csv(os.path.join(storage_path,file_name), index=False)  

## Comparison per category
The following DataFrames are grouped by category.

What to look in here:

- What are the contracts with the lowest Patch Count
- How many tools overlaps
- How good each tool is on the category

In [279]:
df_sorted[df_sorted['Category']=='access_control']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
8,parity_wallet_bug_1.sol,access_control,False,False,False,False,False,False,False,0
9,parity_wallet_bug_2.sol,access_control,False,False,False,True,True,True,False,3
14,unprotected0.sol,access_control,True,False,False,True,True,False,False,3
6,multiowned_vulnerable.sol,access_control,True,False,False,True,True,False,False,3
11,proxy.sol,access_control,True,True,False,True,True,False,False,4
12,rubixi.sol,access_control,False,False,False,True,True,True,True,4
4,incorrect_constructor_name3.sol,access_control,True,True,False,True,True,True,False,5
2,incorrect_constructor_name1.sol,access_control,True,True,False,True,True,True,False,5
0,FibonacciBalance.sol,access_control,True,True,False,True,True,True,False,5
3,incorrect_constructor_name2.sol,access_control,True,True,False,True,True,True,False,5


In [266]:
df_sorted[df_sorted['Category']=='arithmetic']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
18,BECToken.sol,arithmetic,True,False,True,True,True,True,False,5
19,insecure_transfer.sol,arithmetic,True,False,True,True,True,False,True,5
20,integer_overflow_1.sol,arithmetic,True,False,True,True,True,True,False,5
22,integer_overflow_benign_1.sol,arithmetic,True,False,True,True,True,True,False,5
25,integer_overflow_mul.sol,arithmetic,True,False,True,True,True,False,True,5
26,integer_overflow_multitx_multifunc_feasible.sol,arithmetic,True,False,True,True,True,True,False,5
31,token.sol,arithmetic,True,False,True,True,True,True,True,6
29,overflow_single_tx.sol,arithmetic,True,False,True,True,True,True,True,6
28,overflow_simple_add.sol,arithmetic,True,False,True,True,True,True,True,6
27,integer_overflow_multitx_onefunc_feasible.sol,arithmetic,True,False,True,True,True,True,True,6


In [267]:
df_sorted[df_sorted['Category']=='bad_randomness']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
33,blackjack.sol,bad_randomness,True,False,False,True,True,False,False,3
40,smart_billions.sol,bad_randomness,True,False,False,True,False,True,False,3
39,random_number_generator.sol,bad_randomness,True,False,False,True,True,False,False,3
34,etheraffle.sol,bad_randomness,True,False,False,True,True,True,False,4
37,lucky_doubler.sol,bad_randomness,True,True,False,True,True,True,False,5
38,old_blockhash.sol,bad_randomness,True,True,False,True,True,True,False,5
36,lottery.sol,bad_randomness,True,True,False,True,True,False,True,5
35,guess_the_random_number.sol,bad_randomness,True,True,False,True,True,True,False,5


In [268]:
df_sorted[df_sorted['Category']=='denial_of_service']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
43,dos_number.sol,denial_of_service,False,False,False,True,True,False,False,2
42,dos_address.sol,denial_of_service,False,False,False,True,True,False,False,2
44,dos_simple.sol,denial_of_service,False,False,False,True,True,False,True,3
46,send_loop.sol,denial_of_service,True,True,False,True,True,False,False,4
45,list_dos.sol,denial_of_service,True,False,False,True,True,True,True,5
41,auction.sol,denial_of_service,True,True,False,True,True,True,False,5


In [269]:
df_sorted[df_sorted['Category']=='front_running']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
47,ERC20.sol,front_running,True,False,False,True,True,False,False,3
48,FindThisHash.sol,front_running,True,True,False,True,True,True,False,5
49,eth_tx_order_dependence_minimal.sol,front_running,True,True,False,True,True,True,False,5
50,odds_and_evens.sol,front_running,True,True,False,True,True,True,True,6


In [270]:
df_sorted[df_sorted['Category']=='other']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
52,name_registrar.sol,other,True,False,False,True,True,False,False,3
53,open_address_lottery.sol,other,True,True,False,False,True,True,False,4
51,crypto_roulette.sol,other,True,True,False,False,True,True,False,4


In [271]:
df_sorted[df_sorted['Category']=='reentrancy']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
84,spank_chain_payment.sol,reentrancy,False,True,True,False,True,False,False,3
71,0xbe4041d55db380c5ae9d4a9b9703f1ed4e7e3888.sol,reentrancy,True,True,False,True,True,True,True,6
60,0x7541b76cb60f4c60af330c208b0623b7f54bf615.sol,reentrancy,True,True,False,True,True,True,True,6
59,0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839.sol,reentrancy,True,True,True,True,True,True,False,6
58,0x561eac93c92360949ab1f1403323e6db345cbf31.sol,reentrancy,True,True,True,False,True,True,True,6
57,0x4e73b32ed6c35f570686b89848e5f39f20ecc106.sol,reentrancy,True,True,True,False,True,True,True,6
56,0x4320e6f8c05b27ab4707cd1f6d5ce6f3e4b3a5a1.sol,reentrancy,True,True,True,False,True,True,True,6
55,0x23a91059fdc9579a9fbd0edc5f2ea0bfdb70deb4.sol,reentrancy,True,True,True,False,True,True,True,6
54,0x01f8c4e3fa3edeb29e514cba738d87ce8c091d3f.sol,reentrancy,True,True,True,False,True,True,True,6
61,0x7a8721a9d64c74da899424c1b52acbf58ddc9782.sol,reentrancy,True,True,True,False,True,True,True,6


In [272]:
df_sorted[df_sorted['Category']=='short_addresses']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
85,short_address_example.sol,short_addresses,True,False,False,True,True,False,False,3


In [273]:
df_sorted[df_sorted['Category']=='short_addresses']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
85,short_address_example.sol,short_addresses,True,False,False,True,True,False,False,3


In [274]:
df_sorted[df_sorted['Category']=='time_manipulation']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
90,timed_crowdsale.sol,time_manipulation,True,False,False,True,True,False,False,3
89,roulette.sol,time_manipulation,True,True,False,True,True,True,False,5
86,ether_lotto.sol,time_manipulation,True,True,False,True,True,True,False,5
87,governmental_survey.sol,time_manipulation,True,True,False,True,True,False,True,5
88,lottopollo.sol,time_manipulation,True,True,False,True,True,True,True,6


In [275]:
df_sorted[df_sorted['Category']=='unchecked_low_level_calls']

,Contract,Category,SmartShield,sGuard,Aroc,Elysium,TIPS,SmartFix,sGuardPlus,PatchCount
93,0x19cf8481ea15427a98ba3cdd6d9e14690011ab10.sol,unchecked_low_level_calls,False,False,True,True,True,True,False,4
107,0x663e4229142a27f00bafb5d087e1e730648314c3.sol,unchecked_low_level_calls,False,False,True,True,True,True,False,4
95,0x39cfd754c85023648bf003bea2dd498c5612abfa.sol,unchecked_low_level_calls,True,True,True,True,True,False,False,5
120,0xb0510d68f210b7db66e8c7c814f22680f2b8d1d6.sol,unchecked_low_level_calls,True,True,True,False,True,False,True,5
119,0xa46edd6a9a93feec36576ee5048146870ea2c3ae.sol,unchecked_low_level_calls,True,True,True,True,True,False,False,5
118,0xa1fceeff3acc57d257b917e30c4df661401d6431.sol,unchecked_low_level_calls,True,True,True,True,True,False,False,5
115,0x8fd1e427396ddb511533cf9abdbebd0a7e08da35.sol,unchecked_low_level_calls,True,True,True,True,True,False,False,5
102,0x524960d55174d912768678d8c606b4d50b79d7b1.sol,unchecked_low_level_calls,True,True,True,True,True,False,False,5
101,0x4b71ad9c1a84b9b643aa54fdd66e2dec96e8b152.sol,unchecked_low_level_calls,True,True,True,True,True,False,False,5
99,0x4051334adc52057aca763453820cb0e045076ef3.sol,unchecked_low_level_calls,True,True,True,True,True,False,False,5
